In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense,Input
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# from tensorflow.keras.models import Model
print("✅success")

✅success


In [17]:
df = pd.read_csv("C:/Users/rhyth/ML/Py-DS-ML-Bootcamp-master/spam_email_addresses/data/emails_spam_ham.csv")
df

,email_address,label
0,luckyduck880@shaw.ca,ham
1,coder221@freenet.de,ham
2,hao_007zpy2@instantreward321@vipoffers.co,ham
3,meera3@yahoo.com,ham
4,guitarheroXm6a4@superwin.co,spam
...,...,...
9995,isabel9995@orange.fr,spam
9996,pranav9996@virginmedia.com,spam
9997,mohit_cool9997@posta.it,spam
9998,shopaholic019998@numericable.fr,spam


In [19]:
df.head()

,email_address,label
0,luckyduck880@shaw.ca,ham
1,coder221@freenet.de,ham
2,hao_007zpy2@instantreward321@vipoffers.co,ham
3,meera3@yahoo.com,ham
4,guitarheroXm6a4@superwin.co,spam


In [21]:
df.describe()

,email_address,label
count,10000,10000
unique,10000,2
top,luckyduck880@shaw.ca,spam
freq,1,9370


In [23]:
# df.columns = df.columns.str.strip()
df['label'] = df['label'].map({'ham':0,'spam':1})
df.head()

,email_address,label
0,luckyduck880@shaw.ca,0
1,coder221@freenet.de,0
2,hao_007zpy2@instantreward321@vipoffers.co,0
3,meera3@yahoo.com,0
4,guitarheroXm6a4@superwin.co,1


In [25]:


MAX_WORDS = 1000
MAX_LEN = 30

tokenizer = Tokenizer(num_words = MAX_WORDS, char_level=False,oov_token = "<OOV>")
tokenizer.fit_on_texts(df['email_address'])

sequences = tokenizer.texts_to_sequences(df['email_address'])
padded_sequences = pad_sequences(sequences, maxlen = MAX_LEN, padding = 'post')

In [27]:
EMBEDDING_DIM = 32

inputs = Input(shape=(MAX_LEN,))
x = Embedding(MAX_WORDS, EMBEDDING_DIM)(inputs)
x = LSTM(64, name="lstm_layer")(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(padded_sequences, df['label'], epochs=5, batch_size=32, verbose=1)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9346 - loss: 0.2453
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9370 - loss: 0.2362
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9370 - loss: 0.2356
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9370 - loss: 0.2356
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9370 - loss: 0.2358


In [33]:
_ = model.predict(padded_sequences[:1])
emb_model = Model(inputs=model.input,
                  outputs=model.get_layer("lstm_layer").output)
X_embed = emb_model.predict(padded_sequences)
y = df['label'].values

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_embed, y, test_size=0.2, random_state=42)

In [37]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [39]:
y_pred = clf.predict(X_test)
print(" Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))

 Logistic Regression Accuracy: 0.938


In [41]:
import joblib
model.save("lstm_model.h5")
joblib.dump(clf, "logistic_model.joblib")
joblib.dump(tokenizer, "tokenizer.joblib")

['tokenizer.joblib']

In [43]:
from keras.models import load_model
loaded_lstm = load_model("lstm_model.h5")
loaded_clf = joblib.load("logistic_model.joblib")
loaded_tokenizer = joblib.load("tokenizer.joblib")

In [51]:
embedding_model = Model(inputs=loaded_lstm.input,
                       outputs=loaded_lstm.get_layer("lstm_layer").output
)

email = ["freeprize@lottery.com"]
seq = tokenizer.texts_to_sequences(email)
pad = pad_sequences(seq,maxlen = 30, padding='post')
X_embed = embedding_model.predict(pad)
pred = clf.predict(X_embed)


label = "spam" if pred[0] ==1 else "ham"
print("Prediction: ", label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
Prediction:  spam
